In [47]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [69]:
df_train = pd.read_csv("data/final/final_training.csv")
df_train = df_train[df_train['pot_doc'] != '[]']
X_train_df=pd.DataFrame(data = np.array(df), columns = df.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_train_df = pd.DataFrame(data = np.array(df['documentid']), columns = ['documentid'])
X_train_df

df_train = pd.read_csv("data/final/final_test.csv")
X_test_df=pd.DataFrame(data = np.array(df), columns = df.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_test_df = pd.DataFrame(data = np.array(df['documentid']), columns = ['documentid'])


In [86]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2)

            #+ (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_test_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [87]:

# dfs = pd.DataFrame()
# ss = pd.DataFrame()

# dfs['result'] = ss.apply(lambda row: (row['documentid'] in row['pot_doc']), axis=1)
# print(dfs)
# num_trues = dfs['result'].value_counts()[True]
# print(num_trues)
def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    y = trial.suggest_float('y', 0, 10)
    z = trial.suggest_float('z', 0, 1)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,y,z,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}



[I 2023-01-29 07:17:09,814] A new study created in memory with name: no-name-7b622970-115e-479f-a9fb-df62c9461029
[I 2023-01-29 07:17:10,011] Trial 0 finished with value: 0.16801619433198378 and parameters: {'x': 1.9157405090737178, 'y': 2.3110454523936683, 'z': 0.09760513623840483}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:17:10,208] Trial 1 finished with value: 0.16801619433198378 and parameters: {'x': 2.1758086894997106, 'y': 5.103961274643098, 'z': 0.6819490860624772}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:17:10,413] Trial 2 finished with value: 0.16801619433198378 and parameters: {'x': 3.357929782695578, 'y': 0.6204379311787345, 'z': 0.5228266781609568}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:17:10,617] Trial 3 finished with value: 0.16801619433198378 and parameters: {'x': 5.779616915185361, 'y': 3.1910921116166167, 'z': 0.9177136138253703}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-2

{'x': 1.9157405090737178, 'y': 2.3110454523936683, 'z': 0.09760513623840483}

In [88]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['y'], 0)

real = list(y_test_df['documentid'])

print(len(res), len(real))

494 494


In [89]:
def calculate_percentage(list_a, list_b):
    common_elements = set(list_a) & set(list_b)
    percentage = (len(common_elements) / len(list_a)) * 100
    return percentage
print(calculate_percentage(res,real))

84.21052631578947
